Esports is a type of sport that elevates video games competitions to "competitive level". Esports is a intellectural confrontation sport between people using electronic equipment as sports equipment. More than just entertainment, esports requires participants to possess a high level of thinking skills, reflexes, coordination and good team spirits. In esports, participants are usually professional players with high gaming skills who compete for prize money by signing up with specific teams and participating in esports tournaments on a regular basis. 

## Landscape of E-Sports

Video game culture has always included organized competitions, although they were mostly between amateurs until the late 2000s, when professional players began to participate and live broadcasting of these events became quite popular. Esports had become a major force in the video game business by the 2010s, when a large number of game developers actively designed and funded tournaments and other events.


In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [79]:
esports = pd.read_csv("../data/ESport_Earnings.csv", encoding='latin-1')
# general = pd.read_csv("../data/GeneralEsportData.csv")

In [80]:
# There are some "Releaseyear" data invalid, so we need to remove them
esports = esports[esports['Releaseyear'] >= 1980]

In [81]:
# esports.describe()

## Games differed By Genre

### Number of Games Released per Year by Genre

In [82]:
df_count = esports.groupby(['Releaseyear', 'Genre']).size().reset_index(name='Count')

fig = go.Figure()
genres = df_count['Genre'].unique()

for genre in genres:
    fig.add_trace(
        go.Scatter(
            x=df_count[df_count['Genre'] == genre]['Releaseyear'],
            y=df_count[df_count['Genre'] == genre]['Count'],
            mode='lines+markers',
            name=genre
        )
    )

dropdown_buttons = [
    {
        'label': 'All Genres',
        'method': 'update',
        'args': [{'visible': [True] * len(genres)},
                 {'title': 'Number of Games Released per Year by Genre'}]
    }
]

for i, genre in enumerate(genres):
    # True for the selected genre, False for the rest
    visibility = [False] * len(genres)
    visibility[i] = True

    dropdown_button = {
        'label': genre,
        'method': 'update',
        'args': [{'visible': visibility},
                 {'title': f'Number of Games Released per Year: {genre}'}]
    }
    dropdown_buttons.append(dropdown_button)

fig.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ],
    title='Number of Games Released per Year by Genre',
    xaxis=dict(title='Release Year', range=[1980, 2021]),
    yaxis=dict(title='Count'),
    plot_bgcolor='white'
)

fig.write_html("../picture/game_release.html")
fig.show()

### Genre vs. Prize Money

How does the prize money vary across different game genres?


In [83]:
genre_money = esports.groupby('Genre')['TotalMoney'].sum()
genre_money = pd.DataFrame(genre_money).round(2)
genre_money = genre_money.reset_index()
genre_money = genre_money.rename(columns={'TotalMoney': 'Total Amount of Money Earned'})
genre_money = genre_money.sort_values('Total Amount of Money Earned', ascending=False)
genre_money['Total Amount of Money Earned'] = genre_money['Total Amount of Money Earned'].apply(lambda x: "{:,.2f}".format(x))

In [84]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(genre_money.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[genre_money.Genre, genre_money['Total Amount of Money Earned']],
               fill_color='lavender',
               align='left'))
])
fig.write_html("../picture/game_money.html")
fig.show()

## Performance of different games in E-Sports

### Popularity of different games 

Which game has the highest number of players, tournaments and total prize money??


In [85]:
players = esports.groupby('GameName')['PlayerNo'].sum().sort_values(ascending=False).head(15)
players = pd.DataFrame(players).reset_index()


tournaments = esports.groupby('GameName')['TournamentNo'].sum()
tournaments = pd.DataFrame(tournaments).reset_index()


money = esports.groupby('GameName')['TotalMoney'].sum()
money = pd.DataFrame(money).reset_index()

game = pd.merge(players, tournaments, on='GameName', how='inner')
game = pd.merge(game, money, on='GameName', how='inner')

In [86]:
fig = go.Figure()

for g in game['GameName'].unique():
    game_data = game[game['GameName'] == g]

    fig.add_trace(
        go.Bar(x=['Number of Players'], y=game_data['PlayerNo'], name='Number of Players',
               text=f"{int(game_data['PlayerNo'].iloc[0]):,}" ,
               visible=(g==game['GameName'].unique()[0]))
    )

    fig.add_trace(
        go.Bar(x=['Number of Tournaments'], y=game_data['TournamentNo'], 
               name='Number of Tournaments', 
               text=f"{int(game_data['TournamentNo'].iloc[0]):,}",
               visible=(g==game['GameName'].unique()[0]))
    )

    fig.add_trace(
        go.Bar(
            x=['Total Money'], 
            y=game_data['TotalMoney'] , 
            name='Total Money', 
            visible=(g==game['GameName'].unique()[0]),
            text=f"${int(game_data['TotalMoney'].iloc[0]):,}"
    )
)

titles = {g: f"Data summarization until 2020 for game {g}" for g in game['GameName'].unique()}

updatemenus = [{
    'buttons': [
        {
            'method': 'update',
            'label': g,
            'args': [{'visible': [(g==game) for game in game['GameName'].unique() for _ in range(3)]},
                     {'title': titles[g]}]
        } for g in game['GameName'].unique()
    ],
    'direction': 'down',
    'showactive': True,
    'x': 1.15,
    'xanchor': 'right',
    'y': 1.15,
    'yanchor': 'top'
}]

fig.update_layout(
    updatemenus=updatemenus,
    title=titles[game['GameName'].unique()[0]],
    yaxis=dict(
        type='log',
        range=[0,8.3],
        showgrid=False,
        zeroline=True,
        zerolinecolor='black',
        showline=True,
        linecolor='black'
    ),
    xaxis=dict(
        showgrid=False,
        zeroline=True,
        zerolinecolor='black',
        showline=True,
        linecolor='black'
    ),
    plot_bgcolor='white' 
)
fig.write_html("../picture/each_game.html")
fig.show()

### Behavior of different region in Esports 

Which country has the highest earnings in esports?


In [87]:
country_earnings = esports.groupby('Top_Country')['Top_Country_Earnings'].sum().sort_values(ascending=False)
print("The country with the highest earnings in E-Sports is:", country_earnings.index[0])

The country with the highest earnings in E-Sports is: United States


In [88]:
country_earnings = pd.DataFrame(country_earnings)
country_earnings.reset_index(inplace=True)

In [89]:
fig = px.choropleth(country_earnings,
                    locations='Top_Country', 
                    locationmode='country names',
                    color='Top_Country_Earnings',
                    hover_name='Top_Country',  
                    color_continuous_scale=px.colors.sequential.Plasma)

fig.update_layout(title_text='Global eSports Earnings by Country (from 1980 to 2020)')
fig.update_layout(coloraxis_colorbar_title='Top Country Earnings ($)')

fig.write_html("../picture/global_earnings.html")
fig.show()

In [10]:
# players = esports.groupby('GameName')['PlayerNo'].sum().sort_values(ascending=False)
# tournaments = esports.groupby('GameName')['TournamentNo'].sum().sort_values(ascending=False)

In [11]:
# top_15_players = players.head(15)
# other_players = players.iloc[15:].sum()
# top_15_players['Other'] = other_players
# print("The game which has the highest number of players is:", players.index[0])

# print()
# plt.figure(figsize=(10, 8))
# plt.pie(top_15_players, labels=top_15_players.index, autopct='%1.1f%%')
# plt.title('Top 15 Games that have Highest Number of Players (Including Other)', fontsize = 15)
# plt.axis('equal')  
# plt.show()

In [12]:
# top_15_tournaments = tournaments.head(15)
# other_tournaments_sum = tournaments.iloc[15:].sum()
# top_15_tournaments['Other'] = other_tournaments_sum
# print("The game which has the highest number of tournaments is:", tournaments.index[0])
# print()
# plt.figure(figsize=(10, 8))
# plt.pie(top_15_tournaments, labels=top_15_tournaments.index, autopct='%1.1f%%')
# plt.title('Top 15 Games that have Highest Number of Tournaments (Including Other)', fontsize = 15)
# plt.axis('equal')  
# plt.show()

In [13]:
# money = esports.groupby('GameName')['TotalMoney'].sum().sort_values(ascending=False)
# print("The game with the highest total prize money is:", money.index[0])

In [14]:
# top_money = money.head(10)
# other_money = money.iloc[10:].sum()
# top_money['Other'] = other_money

# explode = [0.05] * len(top_money)
# plt.figure(figsize=(10, 8))
# circle = plt.Circle((0,0),0.7, color = 'white')
# plt.pie(top_money, labels = top_money.index, autopct = '%1.1f%%', explode = explode)
# plt.title('Top 10 Games that have Highest Prize Money', fontsize = 15)
# p = plt.gcf()
# p.gca().add_artist(circle)
# plt.show()

In [15]:
# plt.figure(figsize=(12, 8))
# (genre_money / 1e6).sort_values().plot(kind='barh')

# plt.title('Total Prize Money by Game Genre (in millions of $)')
# plt.xlabel('Total Prize Money (in millions of $)')
# plt.ylabel('Game Genre')

# ax = plt.gca()
# ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:.0f}".format(x)))
# ax.set_axisbelow(True)
# plt.grid(axis='x', linestyle='--', color='gray', alpha=0.7)

# plt.show()

In [19]:
# grouped = esports.groupby(['Genre', 'Top_Country']).size().reset_index(name='Count')

# genre_country = grouped.loc[grouped.groupby('Top_Country')['Count'].idxmax()]

In [20]:
# fig = px.choropleth(genre_country,
#                     locations='Top_Country',
#                     locationmode='country names',
#                     color='Genre',
#                     hover_name='Top_Country',
#                     hover_data=['Genre', 'Count'],
#                     title='Most Proficient eSports Genre by Country')

# fig.update_layout(legend_title_text='eSports Genre')

# fig.show()

In [21]:
# data = pd.merge(esports, general, left_on=['GameName', 'Genre', 'Releaseyear'], 
#                 right_on=['Game','Genre', 'ReleaseDate'], how='inner')
# data = data.drop(['IdNo', 'Game', 'ReleaseDate'], axis = 1)
# data.head()